## Loading Dataset

In [1]:
import numpy as np

In [2]:
X_fn = "./data/combine_astii_astiii_filter_all_smoothing_van_enterococcus_norm.npy"
y_fn = "./data/y_astii_astiii_filter_all_van_enterococcus.npy"
patient_fn = "./data/patient_astii_astiii_filter_all_van_enterococcus.npy"
X_train_raw = np.load(X_fn,allow_pickle=True)
y_train_raw = np.load(y_fn,allow_pickle=True)
patient_train_raw = np.load(patient_fn,allow_pickle=True)

classnames=['VRE','VSE']



In [3]:
if not isinstance(X_train_raw, np.ndarray):
    X_train_raw = np.array(X_train_raw, dtype=np.float32)
elif X_train_raw.dtype != np.float32:
    X_train_raw = X_train_raw.astype(np.float32)

### Model Setting

In [4]:
# model
from model import Variant_LeNet,ResNet,RamanNet,ViT
from tqdm import tqdm
import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
from losses import FocalLoss
from losses import AdaptiveProxyAnchorLoss
from imblearn.metrics import specificity_score
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    auc,
    roc_curve,
)
from pytorchtools import EarlyStopping
import torch.optim as optim
# Plot
from plot import plot_VRE_VSE_ROC_curve, plot_heatmap,plot_tsne_interactive_html
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.metrics import confusion_matrix
from config import ORDER, STRAINS
import math

# Training setting
n_classes = 2
epochs = 300 
batch_size = 256
num = 10
num_workers = 7


C_total = np.zeros((n_classes, n_classes))
C = np.zeros((n_classes, n_classes))
C_new = np.zeros((n_classes, n_classes))

2025-07-22 16:18:25.162824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-22 16:18:25.162853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-22 16:18:25.163551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## LeNet + CE

In [5]:


class Solver:
    def __init__(
        self,
        i,
        dataloaders,
        model,
        model_path,
        device,
        n_classes,
        alpha,
        gpu=-1,
    ):
        self.i = i
        self.dataloaders = dataloaders
        self.device = device
        self.net = model
        self.net = self.net.to(self.device)
        self.n_classes = n_classes
        self.criterion = nn.CrossEntropyLoss(weight=alpha).cuda()
        self.get_num_labeled_class = 2
        self.optimizer = optim.Adam(self.net.parameters(),lr = 0.001)
        self.model_path = model_path


    def iterate(self, epoch, phase):
        if phase == "train":
            self.net.train()
        else:
            self.net.eval()

            
        dataloader = self.dataloaders[phase]
        total_loss = 0
        correct = 0
        total = 0
            
        for batch_idx, (inputs, targets,patients) in enumerate(dataloader):
            inputs, targets,patients = Variable(inputs).to(self.device), Variable(targets.long()).to(self.device), patients
            out,emb= self.net(inputs)
            
            loss = self.criterion(out, targets)
            
            outputs = torch.argmax(out.detach(), dim=1)
            
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            total_loss += loss.item()
            total += targets.size(0)
            correct += outputs.eq(targets).cpu().sum().item()

        total_loss /= (batch_idx + 1)
        total_acc = correct / total

                
        print("\nepoch %d: %s average loss: %.3f | acc: %.2f%% (%d/%d)"
                % (epoch + 1, phase, total_loss, 100.0 * total_acc, correct, total))

        return total_loss,total_acc,out,targets
            

    def train(self, epochs):
        best_loss = float("inf")
        best_acc = 0
        epoch_breaks_classifer = 0
        train_losses = []
        train_acces = [] 
        test_losses = []
        test_acces = [] 


        early_stopping_classifier = EarlyStopping(patience=25, mode='acc', verbose=True)

        for epoch in tqdm(range(epochs)):
            train_loss,train_acc,out,targets =self.iterate(epoch, "train")

            train_losses.append(train_loss)
            train_acces.append(train_acc)

            with torch.no_grad():
                
                test_loss,test_acc,out,targets= self.iterate(epoch, "test")
                    
                test_losses.append(test_loss)
                test_acces.append(test_acc)

                if test_acc > best_acc:
                    best_acc = test_acc
                    checkpoint = {
                        "epoch": epoch,
                         "train_acc":train_acc,
                        "test_acc":test_acc,
                        "test_loss": test_loss,
                        "state_dict": self.net.state_dict(),
                    }
                    print("best test acc found")
                    torch.save(checkpoint, self.model_path)

                early_stopping_classifier(test_acc)

                epoch_breaks_classifer+= 1

                if early_stopping_classifier.early_stop:
                    break  # 停止訓練
        
        if not early_stopping_classifier.early_stop:
            epoch_breaks_classifer = epochs
        

    def test_iterate(self, phase):
        self.net.eval()
        dataloader = self.dataloaders[phase]
        y_pred = []
        y_true = []
        y_pred_prob = []
        
        features_out = []
        features_combine = []
        targets_combine = []
        patient_ids = []
        
        with torch.no_grad():
            for batch_idx, (inputs, targets,patients) in enumerate(dataloader):
                inputs, targets ,patients = Variable(inputs).to('cuda'), Variable(targets.long()).to('cuda'), patients
                out,emb = self.net(inputs)
                outputs = torch.argmax(out.detach(), dim=1)
                outputs_prob = nn.functional.softmax(out.detach(), dim=1)
                y_pred.append(outputs.cpu().numpy())
                y_pred_prob.append(outputs_prob.cpu().numpy())
            
                y_true.append(targets.cpu().numpy())
                
                targets_combine.append(targets.detach().cpu().numpy())
                patient_ids.append(patients)
                features_out.append(out.cpu().numpy())
                features_combine.append(emb.detach().cpu().numpy())

            targets_combine = np.concatenate(targets_combine, axis=0) 
            patient_ids = np.concatenate( patient_ids, axis=0) 
            features_out = np.concatenate(features_out, axis=0)
            features_combine = np.concatenate(features_combine, axis=0)
        
        return (
                np.hstack(y_pred),
                np.hstack(y_true),
                np.vstack(y_pred_prob),
                features_out,
                features_combine,
                targets_combine,
                patient_ids,
            )
    def test(self):
        checkpoint = torch.load(self.model_path)
        epoch = checkpoint["epoch"]
        train_acc = checkpoint["train_acc"]
        test_acc = checkpoint["test_acc"]
        self.net.load_state_dict(checkpoint["state_dict"])
        print("load model at epoch {}, with test acc : {:.3f}".format(epoch+1, test_acc))

        
        _, _ ,_,train_out,train_combined,train_targets,train_patient_ids = self.test_iterate("train")
        y_pred, y_true ,y_pred_prob,out,combined,targets, test_patient_ids,= self.test_iterate("test")
        
        print("total", accuracy_score(y_true, y_pred))
        for i in range(self.n_classes):
            idx = y_true == i
            print("class", i, accuracy_score(y_true[idx], y_pred[idx]))

        return (
            confusion_matrix(y_true, y_pred),
            y_true,
            y_pred,
            accuracy_score(y_true, y_pred),
            y_pred_prob,
            train_acc,
            train_targets,
            targets,
            train_combined,
            combined,
            train_patient_ids,
            test_patient_ids
        )

In [6]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

# metrics
lenet_train_avg_accuracy = []
lenet_avg_accuracy = []
lenet_avg_roc = []

sss = StratifiedKFold(n_splits=10, shuffle=True,random_state=1)
for fold, (train_index, test_index) in enumerate(sss.split(X_train_raw, y_train_raw)):
    X_train, X_test = X_train_raw[train_index], X_train_raw[test_index]
    y_train, y_test = y_train_raw[train_index], y_train_raw[test_index]
    patient_train ,patient_test = patient_train_raw[train_index],patient_train_raw[test_index]

    dl_tr = spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = Variant_LeNet(in_channels=1, out_channels=n_classes)

    model_path = f"best_variant_lenet_model_{fold}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights
    solver = Solver(
                fold,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(fold + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    lenet_train_avg_accuracy.append(train_acc)
    lenet_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"variant_lenet_best_test_accuracy_combined_train",f"variant_lenet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"variant_lenet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"variant_lenet_low_test_accuracy_combined_train",f"variant_lenet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"variant_lenet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    lenet_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_VRE_VSE_ROC_curve(f"variant_lenet_{fold}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"variant_lenet_{fold}_heatmap", cm,class_names=classnames)

plot_heatmap(f"variant_lenet_average_heatmap", C_total,class_names=classnames)


1


  0%|          | 0/300 [00:00<?, ?it/s]


epoch 1: train average loss: 0.482 | acc: 75.44% (2872/3807)


  0%|          | 1/300 [00:00<02:37,  1.90it/s]


epoch 1: test average loss: 0.852 | acc: 67.38% (285/423)
best test acc found

epoch 2: train average loss: 0.305 | acc: 87.37% (3326/3807)


...


epoch 49: test average loss: 0.092 | acc: 97.87% (414/423)
EarlyStopping counter: 24/25 (best: 0.9905)

epoch 50: train average loss: 0.040 | acc: 100.00% (3807/3807)


 16%|█▋        | 49/300 [00:16<01:22,  3.03it/s]


epoch 50: test average loss: 0.095 | acc: 97.64% (413/423)
EarlyStopping counter: 25/25 (best: 0.9905)
🔴 Early stopping triggered
load model at epoch 25, with test acc : 0.991



/tmp/ipykernel_463633/474861799.py:156: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



total 0.9905437352245863
class 0 0.9928057553956835
class 1 0.9894366197183099
                            0
Accuracy               0.9905
Recall       [0.9928, 0.9894]
Specificity  [0.9894, 0.9928]
Precision    [0.9787, 0.9965]
F1 Score     [0.9857, 0.9929]


In [7]:
print("max test acc:", np.max(lenet_avg_accuracy))
print("min test acc:", np.min(lenet_avg_accuracy))

print("train mean:", np.mean(lenet_train_avg_accuracy))
print("train std:", np.std(lenet_train_avg_accuracy))

    
print("mean:", np.mean(lenet_avg_accuracy))
print("std:", np.std(lenet_avg_accuracy))

    
print("auc mean:", np.mean(lenet_avg_roc))
print("auc std:", np.std(lenet_avg_roc))
    

max test acc: 1.0
min test acc: 0.9858156028368794
train mean: 0.9992119779353823
train std: 0.0015316395836210534
mean: 0.9914893617021276
std: 0.004121890253939177
auc mean: 0.9979037961495955
auc std: 0.00224754106143297


## ResNet + CE

In [8]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

resnet_train_avg_accuracy = []
resnet_avg_accuracy = []
resnet_avg_roc = []

sss = StratifiedKFold(n_splits=10, shuffle=True,random_state=1)
for fold, (train_index, test_index) in enumerate(sss.split(X_train_raw, y_train_raw)):
    X_train, X_test = X_train_raw[train_index], X_train_raw[test_index]
    y_train, y_test = y_train_raw[train_index], y_train_raw[test_index]
    patient_train ,patient_test = patient_train_raw[train_index],patient_train_raw[test_index]

    dl_tr = spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    # setting resnet
    layers = 6
    hidden_size = 100
    block_size = 2
    hidden_sizes = [hidden_size] * layers
    num_blocks = [block_size] * layers
    input_dim = 400
    model = ResNet(
                    hidden_sizes,
                    num_blocks,
                    input_dim=input_dim,
                    in_channels=64,
                    n_classes=2,
                )

    model_path = f"best_resnet_model_{fold}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights
    solver = Solver(
                fold,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(fold + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    resnet_train_avg_accuracy.append(train_acc)
    resnet_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"resnet_best_test_accuracy_combined_train",f"resnet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"resnet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"resnet_low_test_accuracy_combined_train",f"resnet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"resnet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    resnet_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_VRE_VSE_ROC_curve(f"resnet_{fold}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"resnet_{fold}_heatmap", cm,class_names=classnames)

plot_heatmap(f"resnet_average_heatmap", C_total,class_names=classnames)


1


  0%|          | 0/300 [00:00<?, ?it/s]


epoch 1: train average loss: 0.583 | acc: 67.98% (2588/3807)


  0%|          | 1/300 [00:00<03:43,  1.34it/s]


epoch 1: test average loss: 0.809 | acc: 67.38% (285/423)
best test acc found

epoch 2: train average loss: 0.437 | acc: 83.87% (3193/3807)


...


epoch 60: test average loss: 0.270 | acc: 94.80% (401/423)
EarlyStopping counter: 24/25 (best: 0.9764)

epoch 61: train average loss: 0.007 | acc: 99.89% (3803/3807)


 20%|██        | 60/300 [00:38<02:32,  1.57it/s]


epoch 61: test average loss: 0.130 | acc: 96.45% (408/423)
EarlyStopping counter: 25/25 (best: 0.9764)
🔴 Early stopping triggered
load model at epoch 36, with test acc : 0.976



/tmp/ipykernel_463633/474861799.py:156: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



total 0.9763593380614657
class 0 0.9712230215827338
class 1 0.9788732394366197
                            0
Accuracy               0.9764
Recall       [0.9712, 0.9789]
Specificity  [0.9789, 0.9712]
Precision    [0.9574, 0.9858]
F1 Score     [0.9643, 0.9823]


In [9]:
print("max test acc:", np.max(resnet_avg_accuracy))
print("min test acc:", np.min(resnet_avg_accuracy))

print("train mean:", np.mean(resnet_train_avg_accuracy))
print("train std:", np.std(resnet_train_avg_accuracy))

    
print("mean:", np.mean(resnet_avg_accuracy))
print("std:", np.std(resnet_avg_accuracy))

    
print("auc mean:", np.mean(resnet_avg_roc))
print("auc std:", np.std(resnet_avg_roc))
    

max test acc: 0.9952718676122931
min test acc: 0.9692671394799054
train mean: 0.9959810874704491
train std: 0.0037167213808754736
mean: 0.9810874704491725
std: 0.00845794034042475
auc mean: 0.9941345865519209
auc std: 0.003809990624811562


## RamanNet + CE

In [12]:

from datasets_sliding_window import sliding_spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

ramannet_train_avg_accuracy = []
ramannet_avg_accuracy = []
ramannet_avg_roc = []

sss = StratifiedKFold(n_splits=10, shuffle=True,random_state=1)
for fold, (train_index, test_index) in enumerate(sss.split(X_train_raw, y_train_raw)):
    X_train, X_test = X_train_raw[train_index], X_train_raw[test_index]
    y_train, y_test = y_train_raw[train_index], y_train_raw[test_index]
    patient_train ,patient_test = patient_train_raw[train_index],patient_train_raw[test_index]

    dl_tr = sliding_spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = sliding_spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = RamanNet(w_len=50, n_windows=14,n_classes=2)

    model_path = f"best_ramannet_model_{fold}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights
    solver = Solver(
                fold,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(fold + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    ramannet_train_avg_accuracy.append(train_acc)
    ramannet_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"ramannet_best_test_accuracy_combined_train",f"ramannet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"ramannet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"ramannet_low_test_accuracy_combined_train",f"ramannet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"ramannet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    ramannet_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_VRE_VSE_ROC_curve(f"ramannet_{fold}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"ramannet_{fold}_heatmap", cm,class_names=classnames)

plot_heatmap(f"ramannet_average_heatmap", C_total,class_names=classnames)


1


  0%|          | 0/300 [00:00<?, ?it/s]


epoch 1: train average loss: 0.478 | acc: 75.91% (2890/3807)


  0%|          | 1/300 [00:00<01:54,  2.61it/s]


epoch 1: test average loss: 0.689 | acc: 67.38% (285/423)
best test acc found

epoch 2: train average loss: 0.334 | acc: 84.79% (3228/3807)


...


epoch 78: test average loss: 0.060 | acc: 98.11% (415/423)
EarlyStopping counter: 24/25 (best: 0.9905)

epoch 79: train average loss: 0.019 | acc: 99.29% (3780/3807)


 26%|██▌       | 78/300 [00:30<01:27,  2.54it/s]


epoch 79: test average loss: 0.049 | acc: 98.35% (416/423)
EarlyStopping counter: 25/25 (best: 0.9905)
🔴 Early stopping triggered
load model at epoch 54, with test acc : 0.991



/tmp/ipykernel_463633/474861799.py:156: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



total 0.9905437352245863
class 0 0.9928057553956835
class 1 0.9894366197183099
                            0
Accuracy               0.9905
Recall       [0.9928, 0.9894]
Specificity  [0.9894, 0.9928]
Precision    [0.9787, 0.9965]
F1 Score     [0.9857, 0.9929]


In [13]:
print("max test acc:", np.max(ramannet_avg_accuracy))
print("min test acc:", np.min(ramannet_avg_accuracy))

print("train mean:", np.mean(ramannet_train_avg_accuracy))
print("train std:", np.std(ramannet_train_avg_accuracy))

    
print("mean:", np.mean(ramannet_avg_accuracy))
print("std:", np.std(ramannet_avg_accuracy))

    
print("auc mean:", np.mean(ramannet_avg_roc))
print("auc std:", np.std(ramannet_avg_roc))

max test acc: 0.9952718676122931
min test acc: 0.9763593380614657
train mean: 0.9832676648279485
train std: 0.005752561071709994
mean: 0.9903073286052008
std: 0.00500936645399932
auc mean: 0.9980557138071411
auc std: 0.0021731304569534067


## ViT+CE

In [17]:
class Solver:
    def __init__(
        self,
        i,
        dataloaders,
        model,
        model_path,
        device,
        n_classes,
        alpha,
        gpu=-1,
    ):
        self.i = i
        self.dataloaders = dataloaders
        self.device = device
        self.net = model
        self.net = self.net.to(self.device)
        self.n_classes = n_classes
        self.criterion = nn.CrossEntropyLoss(weight=alpha).cuda()
        self.get_num_labeled_class = 2
        self.optimizer = optim.Adam(self.net.parameters(),lr = 0.001)
        self.model_path = model_path


    def iterate(self, epoch, phase):
        if phase == "train":
            self.net.train()
        else:
            self.net.eval()

            
        dataloader = self.dataloaders[phase]
        total_loss = 0
        correct = 0
        total = 0
            
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = Variable(inputs).to(self.device), Variable(targets.long()).to(self.device)
            out,emb= self.net(inputs)
            
            loss = self.criterion(out, targets)
            
            outputs = torch.argmax(out.detach(), dim=1)
            
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            total_loss += loss.item()
            total += targets.size(0)
            correct += outputs.eq(targets).cpu().sum().item()

        total_loss /= (batch_idx + 1)
        total_acc = correct / total

                
        print("\nepoch %d: %s average loss: %.3f | acc: %.2f%% (%d/%d)"
                % (epoch + 1, phase, total_loss, 100.0 * total_acc, correct, total))

        return total_loss,total_acc,out,targets
            

    def train(self, epochs):
        best_loss = float("inf")
        best_acc = 0
        epoch_breaks_classifer = 0
        train_losses = []
        train_acces = [] 
        test_losses = []
        test_acces = [] 


        early_stopping_classifier = EarlyStopping(patience=25, mode='acc', verbose=True)

        for epoch in tqdm(range(epochs)):
            train_loss,train_acc,out,targets =self.iterate(epoch, "train")

            train_losses.append(train_loss)
            train_acces.append(train_acc)

            with torch.no_grad():
                
                test_loss,test_acc,out,targets= self.iterate(epoch, "test")
                    
                test_losses.append(test_loss)
                test_acces.append(test_acc)

                if test_acc > best_acc:
                    best_acc = test_acc
                    checkpoint = {
                        "epoch": epoch,
                         "train_acc":train_acc,
                        "test_acc":test_acc,
                        "test_loss": test_loss,
                        "state_dict": self.net.state_dict(),
                    }
                    print("best test acc found")
                    torch.save(checkpoint, self.model_path)

                early_stopping_classifier(test_acc)

                epoch_breaks_classifer+= 1

                if early_stopping_classifier.early_stop:
                    break  # 停止訓練
        
        if not early_stopping_classifier.early_stop:
            epoch_breaks_classifer = epochs
        

    def test_iterate(self, phase):
        self.net.eval()
        dataloader = self.dataloaders[phase]
        y_pred = []
        y_true = []
        y_pred_prob = []
        
        features_out = []
        features_combine = []
        targets_combine = []
        patient_ids = []
        
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(dataloader):
                inputs, targets = Variable(inputs).to('cuda'), Variable(targets.long()).to('cuda')
                out,emb = self.net(inputs)
                outputs = torch.argmax(out.detach(), dim=1)
                outputs_prob = nn.functional.softmax(out.detach(), dim=1)
                y_pred.append(outputs.cpu().numpy())
                y_pred_prob.append(outputs_prob.cpu().numpy())
            
                y_true.append(targets.cpu().numpy())
                
                targets_combine.append(targets.detach().cpu().numpy())
                features_out.append(out.cpu().numpy())
                features_combine.append(emb.detach().cpu().numpy())

            targets_combine = np.concatenate(targets_combine, axis=0) 
            features_out = np.concatenate(features_out, axis=0)
            features_combine = np.concatenate(features_combine, axis=0)
        
        return (
                np.hstack(y_pred),
                np.hstack(y_true),
                np.vstack(y_pred_prob),
                features_out,
                features_combine,
                targets_combine,
            )
    def test(self):
        checkpoint = torch.load(self.model_path)
        epoch = checkpoint["epoch"]
        train_acc = checkpoint["train_acc"]
        test_acc = checkpoint["test_acc"]
        self.net.load_state_dict(checkpoint["state_dict"])
        print("load model at epoch {}, with test acc : {:.3f}".format(epoch+1, test_acc))

        
        _, _ ,_,train_out,train_combined,train_targets = self.test_iterate("train")
        y_pred, y_true ,y_pred_prob,out,combined,targets= self.test_iterate("test")
        
        print("total", accuracy_score(y_true, y_pred))
        for i in range(self.n_classes):
            idx = y_true == i
            print("class", i, accuracy_score(y_true[idx], y_pred[idx]))

        return (
            confusion_matrix(y_true, y_pred),
            y_true,
            y_pred,
            accuracy_score(y_true, y_pred),
            y_pred_prob,
            train_acc,
            train_targets,
            targets,
            train_combined,
            combined,
        )

In [19]:
from sklearn.model_selection import StratifiedKFold
import torch.utils.data as Data
from plot import plot_tsne
best_test_accuracy = 0
low_test_accuracy = 1

ViT_train_avg_accuracy = []
ViT_avg_accuracy = []
ViT_avg_roc = []

sss = StratifiedKFold(n_splits=10, shuffle=True,random_state=1)
for fold, (train_index, test_index) in enumerate(sss.split(X_train_raw, y_train_raw)):
    X_train, X_test = X_train_raw[train_index], X_train_raw[test_index]
    y_train, y_test = y_train_raw[train_index], y_train_raw[test_index]
    patient_train ,patient_test = patient_train_raw[train_index],patient_train_raw[test_index]

    # Set up dataloaders
    X_train = torch.from_numpy(X_train.astype(np.float32))
    y_train = torch.from_numpy(y_train.astype(np.longlong))


    if isinstance(X_test, np.ndarray):
        X_test = torch.from_numpy(X_test).float()
    else:
        X_test = X_test.float()  # 如果 X_test 已經是張量

    if isinstance(y_test, np.ndarray):
        y_test = torch.from_numpy(y_test).long()
    else:
        y_test = y_test.long()  # 如果 y_test 已經是張量


    dataset_tr = Data.TensorDataset(X_train, y_train)
    dl_tr = Data.DataLoader(dataset_tr, batch_size=batch_size, shuffle = True)
        
        
    dataset_test = Data.TensorDataset(X_test, y_test)
    dl_test = Data.DataLoader(dataset_test, batch_size=batch_size, shuffle = False)
    
    values, counts = np.unique(np.asarray(y_test), return_counts=True)

    dataloaders = {"train": dl_tr, "test": dl_test}

    model = ViT(image_size = (1,400), patch_size = (1, 16), num_classes = n_classes, channels = 1, dim = 64, depth = 3, heads = 6, mlp_dim = 128, dropout = 0.1, emb_dropout = 0.1)

    model_path = f"best_ViT_{fold}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights
    solver = Solver(
                fold,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(fold + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined = solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    ViT_train_avg_accuracy.append(train_acc)
    ViT_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne(f"ViT_best_test_accuracy_combined_train",f"ViT_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets,classnames )
      
        plot_heatmap(f"ViT_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne(f"ViT_low_test_accuracy_combined_train",f"ViT_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets, classnames )
            
        plot_heatmap(f"ViT_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    ViT_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_VRE_VSE_ROC_curve(f"ViT_{fold}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"ViT_{fold}_heatmap", cm,class_names=classnames)

plot_heatmap(f"ViT_average_heatmap", C_total,class_names=classnames)


1


  0%|          | 1/300 [00:00<00:42,  7.02it/s]


epoch 1: train average loss: 0.691 | acc: 58.66% (2233/3807)

epoch 1: test average loss: 0.666 | acc: 60.99% (258/423)
best test acc found


  1%|          | 2/300 [00:00<00:40,  7.35it/s]


epoch 2: train average loss: 0.631 | acc: 66.77% (2542/3807)

epoch 2: test average loss: 0.630 | acc: 72.81% (308/423)
best test acc found


...

 21%|██        | 62/300 [00:07<00:28,  8.49it/s]


epoch 62: train average loss: 0.078 | acc: 96.98% (3692/3807)

epoch 62: test average loss: 0.155 | acc: 92.67% (392/423)
EarlyStopping counter: 24/25 (best: 0.9362)


 21%|██        | 62/300 [00:07<00:29,  8.10it/s]


epoch 63: train average loss: 0.066 | acc: 97.16% (3699/3807)

epoch 63: test average loss: 0.204 | acc: 90.78% (384/423)
EarlyStopping counter: 25/25 (best: 0.9362)
🔴 Early stopping triggered
load model at epoch 38, with test acc : 0.936
total 0.9361702127659575
class 0 0.9136690647482014
class 1 0.9471830985915493
                            0
Accuracy               0.9362
Recall       [0.9137, 0.9472]
Specificity  [0.9472, 0.9137]
Precision    [0.8944, 0.9573]
F1 Score     [0.9039, 0.9522]



/tmp/ipykernel_463633/960659495.py:153: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [20]:
print("max test acc:", np.max(ViT_avg_accuracy))
print("min test acc:", np.min(ViT_avg_accuracy))

print("train mean:", np.mean(ViT_train_avg_accuracy))
print("train std:", np.std(ViT_train_avg_accuracy))

    
print("mean:", np.mean(ViT_avg_accuracy))
print("std:", np.std(ViT_avg_accuracy))

    
print("auc mean:", np.mean(ViT_avg_roc))
print("auc std:", np.std(ViT_avg_roc))

max test acc: 0.9692671394799054
min test acc: 0.9361702127659575
train mean: 0.9654058313632781
train std: 0.015011102154282493
mean: 0.9491725768321515
std: 0.011593569584150552
auc mean: 0.9827552248979814
auc std: 0.007544709848824197


## LeNet + MAPA + Focal

In [21]:
class Solver:
    def __init__(
        self,
        i,
        dataloaders,
        model,
        model_path,
        device,
        n_classes,
        alpha,
        gpu=-1,
    ):
        self.i = i
        self.dataloaders = dataloaders
        self.device = device
        self.net = model
        self.net = self.net.to(self.device)
        self.n_classes = n_classes
        self.criterion_proxy =AdaptiveProxyAnchorLoss(nb_classes=n_classes, sz_embed=256, mrg=0.5, alpha=32,\
                                             nb_proxies=1, scale_margin=1).cuda()
        self.criterion = FocalLoss(alpha=alpha, gamma=2).cuda()
        self.get_num_labeled_class = 2
        self.param_groups = [
            {'params': list(set(model.parameters()).difference(set(model.embedding.parameters())))},
            {'params': model.embedding.parameters() , 'lr':float(0.001) * 1},
            {'params': self.criterion_proxy.mrg,'lr':float(0.001) },
            {'params': self.criterion_proxy.proxies, 'lr':float(0.001) * 100}
        ]
        self.optimizer =torch.optim.AdamW(self.param_groups, lr=0.001, weight_decay = 0.001)
        self.model_path = model_path


    def iterate(self, epoch, phase):
        if phase == "train":
            self.net.train()
        else:
            self.net.eval()

            
        dataloader = self.dataloaders[phase]
        total_loss = 0
        correct = 0
        total = 0
        total_proxy_loss = 0
        total_crossentropy_loss = 0
            
        for batch_idx, (inputs, targets,patients) in enumerate(dataloader):
            inputs, targets,patients = Variable(inputs).to(self.device), Variable(targets.long()).to(self.device), patients
            #print(inputs.shape)
            out,emb= self.net(inputs)
            
            loss1 = self.criterion(out, targets)
            loss2 = self.criterion_proxy(emb, targets)
            loss = loss1 + 0.5 * loss2
            outputs = torch.argmax(out.detach(), dim=1)
            
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                torch.nn.utils.clip_grad_value_(self.net.parameters(), 10)
                torch.nn.utils.clip_grad_value_(self.criterion_proxy.parameters(), 10)


            total_loss += loss.item()
            total_proxy_loss += loss2.item()
            total_crossentropy_loss += loss1.item()
            total += targets.size(0)
            correct += outputs.eq(targets).cpu().sum().item()

        total_loss /= (batch_idx + 1)
        total_acc = correct / total

                
        print("\nepoch %d: %s average loss: %.3f | acc: %.2f%% (%d/%d)"
                % (epoch + 1, phase, total_loss, 100.0 * total_acc, correct, total))

        return total_loss,total_acc,out,targets
            

    def train(self, epochs):
        best_loss = float("inf")
        best_acc = 0
        epoch_breaks_classifer = 0
        train_losses = []
        train_acces = [] 
        test_losses = []
        test_acces = [] 

        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=60)

        early_stopping_classifier = EarlyStopping(patience=25, mode='acc', verbose=True)

        for epoch in tqdm(range(epochs)):
            train_loss,train_acc,out,targets =self.iterate(epoch, "train")

            train_losses.append(train_loss)
            train_acces.append(train_acc)

            with torch.no_grad():
                
                test_loss,test_acc,out,targets= self.iterate(epoch, "test")
                    
                test_losses.append(test_loss)
                test_acces.append(test_acc)

                if test_acc > best_acc:
                    best_acc = test_acc
                    checkpoint = {
                        "epoch": epoch,
                         "train_acc":train_acc,
                        "test_acc":test_acc,
                        "test_loss": test_loss,
                        "state_dict": self.net.state_dict(),
                    }
                    print("best test acc found")
                    torch.save(checkpoint, self.model_path)
                
                self.scheduler.step()

                early_stopping_classifier(test_acc)

                epoch_breaks_classifer+= 1

                if early_stopping_classifier.early_stop:
                    break  # 停止訓練
        
        if not early_stopping_classifier.early_stop:
            epoch_breaks_classifer = epochs
        

    def test_iterate(self, phase):
        self.net.eval()
        dataloader = self.dataloaders[phase]
        y_pred = []
        y_true = []
        y_pred_prob = []
        
        features_out = []
        features_combine = []
        targets_combine = []
        patient_ids = []
        
        with torch.no_grad():
            for batch_idx, (inputs, targets,patients) in enumerate(dataloader):
                inputs, targets ,patients = Variable(inputs).to('cuda'), Variable(targets.long()).to('cuda'), patients
                out,emb = self.net(inputs)
                outputs = torch.argmax(out.detach(), dim=1)
                outputs_prob = nn.functional.softmax(out.detach(), dim=1)
                y_pred.append(outputs.cpu().numpy())
                y_pred_prob.append(outputs_prob.cpu().numpy())
            
                y_true.append(targets.cpu().numpy())
                
                targets_combine.append(targets.detach().cpu().numpy())
                patient_ids.append(patients)
                features_out.append(out.cpu().numpy())
                features_combine.append(emb.detach().cpu().numpy())

            targets_combine = np.concatenate(targets_combine, axis=0) 
            patient_ids = np.concatenate( patient_ids, axis=0) 
            features_out = np.concatenate(features_out, axis=0)
            features_combine = np.concatenate(features_combine, axis=0)
        
        return (
                np.hstack(y_pred),
                np.hstack(y_true),
                np.vstack(y_pred_prob),
                features_out,
                features_combine,
                targets_combine,
                patient_ids,
            )
    def test(self):
        checkpoint = torch.load(self.model_path)
        epoch = checkpoint["epoch"]
        train_acc = checkpoint["train_acc"]
        test_acc = checkpoint["test_acc"]
        self.net.load_state_dict(checkpoint["state_dict"])
        print("load model at epoch {}, with test acc : {:.3f}".format(epoch+1, test_acc))

        
        _, _ ,_,train_out,train_combined,train_targets,train_patient_ids = self.test_iterate("train")
        y_pred, y_true ,y_pred_prob,out,combined,targets, test_patient_ids,= self.test_iterate("test")
        
        print("total", accuracy_score(y_true, y_pred))
        for i in range(self.n_classes):
            idx = y_true == i
            print("class", i, accuracy_score(y_true[idx], y_pred[idx]))

        return (
            confusion_matrix(y_true, y_pred),
            y_true,
            y_pred,
            accuracy_score(y_true, y_pred),
            y_pred_prob,
            train_acc,
            train_targets,
            targets,
            train_combined,
            combined,
            train_patient_ids,
            test_patient_ids
        )

In [22]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

# metrics
lenet_mapa_focal_train_avg_accuracy = []
lenet_mapa_focal_avg_accuracy = []
lenet_mapa_focal_avg_roc = []

sss = StratifiedKFold(n_splits=10, shuffle=True,random_state=1)
for fold, (train_index, test_index) in enumerate(sss.split(X_train_raw, y_train_raw)):
    X_train, X_test = X_train_raw[train_index], X_train_raw[test_index]
    y_train, y_test = y_train_raw[train_index], y_train_raw[test_index]
    patient_train ,patient_test = patient_train_raw[train_index],patient_train_raw[test_index]

    dl_tr = spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = Variant_LeNet(in_channels=1, out_channels=n_classes)

    model_path = f"best_variant_lenet_mapa_focal_model_{fold}.pth"

    class_counts = np.bincount(y_train)
    num_classes = len(class_counts)
    total_samples = len(y_train)

    class_weights = []
    for count in class_counts:
        weight = 1 / (count / total_samples)
        class_weights.append(weight)
    class_weights = torch.FloatTensor(class_weights)
    alpha = class_weights
    solver = Solver(
                fold,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(fold + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    lenet_mapa_focal_train_avg_accuracy.append(train_acc)
    lenet_mapa_focal_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"lenet_mapa_focal_best_test_accuracy_combined_train",f"lenet_mapa_focal_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"lenet_mapa_focal_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"lenet_mapa_focal_low_test_accuracy_combined_train",f"lenet_mapa_focal_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"lenet_mapa_focal_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    lenet_mapa_focal_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_VRE_VSE_ROC_curve(f"lenet_mapa_focal_{fold}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"lenet_mapa_focal_{fold}_heatmap", cm,class_names=classnames)

plot_heatmap(f"lenet_mapa_focal_average_heatmap", C_total,class_names=classnames)


1


  0%|          | 0/300 [00:00<?, ?it/s]


epoch 1: train average loss: 33.095 | acc: 71.58% (2725/3807)


  0%|          | 1/300 [00:00<02:38,  1.88it/s]


epoch 1: test average loss: 25.579 | acc: 67.38% (285/423)
best test acc found

epoch 2: train average loss: 26.034 | acc: 77.65% (2956/3807)


...


epoch 62: test average loss: 24.041 | acc: 97.40% (412/423)
EarlyStopping counter: 24/25 (best: 0.9811)

epoch 63: train average loss: 3.509 | acc: 100.00% (3807/3807)


 21%|██        | 62/300 [00:28<01:48,  2.19it/s]


epoch 63: test average loss: 23.907 | acc: 97.40% (412/423)
EarlyStopping counter: 25/25 (best: 0.9811)
🔴 Early stopping triggered
load model at epoch 38, with test acc : 0.981



/tmp/ipykernel_463633/1054042983.py:175: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



total 0.9810874704491725
class 0 0.9640287769784173
class 1 0.9894366197183099
                            0
Accuracy               0.9811
Recall        [0.964, 0.9894]
Specificity   [0.9894, 0.964]
Precision    [0.9781, 0.9825]
F1 Score       [0.971, 0.986]


In [23]:
print("max test acc:", np.max(lenet_mapa_focal_avg_accuracy))
print("min test acc:", np.min(lenet_mapa_focal_avg_accuracy))

print("train mean:", np.mean(lenet_mapa_focal_train_avg_accuracy))
print("train std:", np.std(lenet_mapa_focal_train_avg_accuracy))

    
print("mean:", np.mean(lenet_mapa_focal_avg_accuracy))
print("std:", np.std(lenet_mapa_focal_avg_accuracy))

    
print("auc mean:", np.mean(lenet_mapa_focal_avg_roc))
print("auc std:", np.std(lenet_mapa_focal_avg_roc))

max test acc: 0.9976359338061466
min test acc: 0.9763593380614657
train mean: 0.9976096664039925
train std: 0.0034975122412619335
mean: 0.9872340425531914
std: 0.006619385342789608
auc mean: 0.9967053682931116
auc std: 0.003119032711097385


In [26]:
import numpy as np

# 將模型名稱與對應數據組成字典，便於迴圈處理
models = {
    "LeNet + MAPA + Focal": lenet_mapa_focal_avg_accuracy,
    "RamanNet + CE": ramannet_avg_accuracy,
    "LeNet + CE": lenet_avg_accuracy,
    "ResNet + CE": resnet_avg_accuracy,
    "ViT + CE": ViT_avg_accuracy
}

# 輸出結果
print("Model Comparison (Mean ± Std):")
for name, acc in models.items():
    mean_acc = np.mean(acc)
    std_acc = np.std(acc)
    print(f"{name:<20}: {mean_acc:.4f} ± {std_acc:.4f}")


Model Comparison (Mean ± Std):
LeNet + MAPA + Focal: 0.9872 ± 0.0066
RamanNet + CE       : 0.9903 ± 0.0050
LeNet + CE          : 0.9915 ± 0.0041
ResNet + CE         : 0.9811 ± 0.0085
ViT + CE            : 0.9492 ± 0.0116
